In [1]:
from snowflake.snowpark import Session
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

import pandas as pd

In [2]:
df =  pd.read_csv('/data/Lumin_Dump_CreditRiskAnalytics_v1.csv')

In [3]:
df.shape

(356255, 108)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 108 entries, RECORD_COUNT to Credit Bureau Requests
dtypes: float64(71), int64(17), object(20)
memory usage: 293.5+ MB


In [6]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [7]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [8]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


,RECORD_COUNT,SK_ID_CURR,PD,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,Credit Bureau Requests
count,356255.000000,356255.000000,356255.000000,356255.000000,356255.000000,3.562550e+05,3.562550e+05,356219.000000,3.559770e+05,356255.000000,...,355205.000000,355205.000000,355205.000000,15709.000000,308687.000000,308687.000000,308687.000000,308687.000000,308687.000000,308687.000000
mean,178128.000000,278128.000000,0.369686,0.138328,0.414316,1.701161e+05,5.877674e+05,27425.560657,5.280200e+05,0.020917,...,0.143452,1.409468,0.100198,952.981412,0.005808,0.006281,0.029995,0.231697,0.304399,1.911564
std,102842.104413,102842.104413,0.224214,0.345244,0.720378,2.235068e+05,3.986237e+05,14732.808190,3.660650e+05,0.013915,...,0.456579,2.577724,0.368259,823.481801,0.079736,0.104250,0.191374,0.855949,0.786915,1.865338
min,1.000000,100001.000000,0.050000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000253,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,89064.500000,189064.500000,0.190000,0.000000,0.000000,1.125000e+05,2.700000e+05,16731.000000,2.340000e+05,0.010006,...,0.000000,0.000000,0.000000,261.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,178128.000000,278128.000000,0.340000,0.000000,0.000000,1.530000e+05,5.002110e+05,25078.500000,4.500000e+05,0.018850,...,0.000000,0.000000,0.000000,742.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,267191.500000,367191.500000,0.490000,0.000000,1.000000,2.025000e+05,7.975575e+05,34960.500000,6.750000e+05,0.028663,...,0.000000,2.000000,0.000000,1550.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,356255.000000,456255.000000,1.000000,1.000000,20.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,...,34.000000,351.000000,24.000000,3983.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [11]:
df['HOUSETYPE_MODE'].value_counts()

HOUSETYPE_MODE
block of flats      175162
specific housing      1761
terraced house        1416
Name: count, dtype: int64

In [10]:
df_snowflake=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_snowflake.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION")

TypeError: field HOUSETYPE_MODE: Cannot merge type <class 'snowflake.snowpark.types.StringType'> and <class 'snowflake.snowpark.types.FloatType'>